In [ ]:
from nmnh_ms_tools.records import Reference
from xmu import EMuAPI

In [ ]:
# Create the EMuAPI object based on params in the TOML file
api = EMuAPI(config_path=r"emurestapi.toml")

# Resolve the DOI using the Reference class from nmnh_ms_tools
ref = Reference("https://doi.org/10.1093/petrology/egu024")

# Insert then display the bibliography record
resp = api.insert("ebibliography", ref.to_emu())
resp.first()